In [0]:
# Pachages needed
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pickle

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, Dropout, MaxPooling1D, Flatten, MaxPooling2D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping

import re, re, csv, math, codecs
import matplotlib.pyplot as plt
%matplotlib inline

! pip install pycm
from pycm import *
from sklearn.metrics import confusion_matrix,classification_report



In [0]:
import pandas as pd
test_data=pd.read_csv('gdrive/My Drive/test_script3.csv')
train_data=pd.read_csv('gdrive/My Drive/train_script3.csv')

In [0]:
train_data = train_data[['lines', 'character']]
test_data = test_data[['lines', 'character']]

In [0]:
train_data.lines = train_data.lines.astype(str)
test_data.lines = test_data.lines.astype(str)

In [0]:
all_data = pd.concat([train_data, test_data]).reset_index()

In [0]:
maxWords = 100000
tokenizer = Tokenizer(num_words=maxWords, split=' ')
tokenizer.fit_on_texts(all_data['lines'].values)

In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(train_data['character'])
le.classes_
labels_train = le.transform(train_data['character']) 


zero = list(le.inverse_transform([0]))
one = list(le.inverse_transform([1]))
two = list(le.inverse_transform([2]))
three = list(le.inverse_transform([3]))
four = list(le.inverse_transform([4]))
five = list(le.inverse_transform([5]))
six = list(le.inverse_transform([6]))

print(zero, 'is encoded as 0')
print(one, 'is encoded as 1')
print(two, 'is encoded as 2')
print(three, 'is encoded as 3')
print(four, 'is encoded as 4')
print(five, 'is encoded as 5')
print(six, 'is encoded as 6')

['Amy'] is encoded as 0
['Bernadette'] is encoded as 1
['Howard'] is encoded as 2
['Leonard'] is encoded as 3
['Penny'] is encoded as 4
['Raj'] is encoded as 5
['Sheldon'] is encoded as 6


In [0]:
le.fit(test_data['character'])
le.classes_
labels_test = le.transform(test_data['character']) 


zero = list(le.inverse_transform([0]))
one = list(le.inverse_transform([1]))
two = list(le.inverse_transform([2]))
three = list(le.inverse_transform([3]))
four = list(le.inverse_transform([4]))
five = list(le.inverse_transform([5]))
six = list(le.inverse_transform([6]))

print(zero, 'is encoded as 0')
print(one, 'is encoded as 1')
print(two, 'is encoded as 2')
print(three, 'is encoded as 3')
print(four, 'is encoded as 4')
print(five, 'is encoded as 5')
print(six, 'is encoded as 6')

['Amy'] is encoded as 0
['Bernadette'] is encoded as 1
['Howard'] is encoded as 2
['Leonard'] is encoded as 3
['Penny'] is encoded as 4
['Raj'] is encoded as 5
['Sheldon'] is encoded as 6


In [0]:
train_data['character'] = labels_train
test_data['character'] = labels_test

In [0]:
data = [train_data, test_data]
all_data = pd.concat(data).reset_index()
all_data = all_data[['lines', 'character']]

In [0]:
maxWords = 100000
tokenizer = Tokenizer(num_words=maxWords, split=' ')
tokenizer.fit_on_texts(all_data['lines'].values)
word_index = tokenizer.word_index
X = tokenizer.texts_to_sequences(all_data['lines'].values)
X = pad_sequences(X)
Y = all_data['character']
lstm_out = 196

In [0]:
len(X[2])

1516

In [0]:
from keras.regularizers import l2
X_train, X_val, Y_train, Y_val  = train_test_split(X[0:1271], Y[0:1271], test_size=0.2, random_state=42)

In [0]:
X_test = X[1271:]
Y_test = Y[1271:]

In [0]:
model = Sequential()
model.add(Embedding(maxWords, 100, input_length = 1516))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(64, kernel_regularizer=l2(0.01), dropout=0.3, recurrent_dropout=0.2, return_sequences=True))) 
model.add(BatchNormalization(momentum=0.99))
model.add(MaxPooling1D(pool_size=1)) 
model.add(Flatten())
model.add(Dense(7,activation='softmax'))
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1516, 100)         10000000  
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 1516, 100)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1516, 128)         84480     
_________________________________________________________________
batch_normalization_1 (Batch (None, 1516, 128)         512       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1516, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 194048)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 7)                

In [0]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
earlyStop=EarlyStopping(monitor="val_loss",verbose=2,mode='auto',patience=20)
checkpointer = ModelCheckpoint(filepath='gdrive/My Drive/weights/biLSTM-{epoch:02d}.hdf5', monitor='val_acc', verbose=1, save_best_only=False, mode='max')

In [0]:
batch_size = 100

In [0]:
history = model.fit(X_train, Y_train, epochs = 200, 
                  batch_size = batch_size,
                    verbose = 1, shuffle=True, 
                    validation_data=(X_val, Y_val),
                   callbacks=[earlyStop,checkpointer])

In [0]:
from keras.models import load_model

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:

model1 = load_model('gdrive/My Drive/weights/biLSTM-58.hdf5')
y_pred = model1.predict_classes(X_test)

y_pred = y_pred.tolist()
y_test = Y_test.tolist()

#! pip install pycm
#from pycm import *
cm = ConfusionMatrix(y_pred,y_test )

confusion_matrix = pd.DataFrame(data=cm.table)
confusion_matrix.columns=['Amy', 'Bernadette', 'Howard', 'Leonard', 'Penny', 'Raj','Sheldon']
confusion_matrix.index = ['Amy', 'Bernadette', 'Howard', 'Leonard', 'Penny', 'Raj', 'Sheldon']
confusion_matrix


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


,Amy,Bernadette,Howard,Leonard,Penny,Raj,Sheldon
Amy,8,6,2,2,4,2,0
Bernadette,3,10,6,1,1,3,0
Howard,3,5,10,2,1,3,0
Leonard,2,3,6,3,7,3,0
Penny,0,3,2,5,13,1,0
Raj,4,1,9,4,1,4,1
Sheldon,1,0,2,2,1,0,18


In [0]:
# The metrics of performance
TP = np.diag(confusion_matrix) 
FP = confusion_matrix.sum(axis=1) - np.diag(confusion_matrix) 
FN = confusion_matrix.sum(axis=0) - np.diag(confusion_matrix)


PRE = (TP)/(TP+FP)
print('precision:', np.mean(PRE))
REC = (TP)/(TP+FN)
print('recall:', np.mean(REC))
F1 = 2*np.mean(REC)*np.mean(PRE)/(np.mean(REC)+np.mean(PRE))
print('F1:', np.mean(F1))


precision: 0.39285714285714285
recall: 0.40398776864942276
F1: 0.3983447173949531


In [0]:
# This is for the error analysis, just briefly describe how sentences that were not classified correclty look like 
mistakes = []
for i in range(len(y_test)):
  if y_test[i] != y_pred[i]:
    mistakes.append(i)

In [0]:
a = pd.read_csv('gdrive/My Drive/test_script3.csv')

In [0]:
# Check the sentences that were missflassied 
for i in range(len(mistakes)):
  idx = mistakes[i]
  print ('--- Text ---')
  print(train_data['lines'][idx]) 
  print('--- Predicted ---')
  print(y_pred[idx])
  print('--- Actual ---')
  print(y_test[idx])
  print('---  ---')